<a href="https://colab.research.google.com/github/MatteoOnger/NLP_Project/blob/main/NLP_EED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP Project: Enron Email Dataset**

*   **Author:** Matteo Onger
*   **Date:** December 2024

**Dataset documentation**:
*   [Enron email dataset](https://www.kaggle.com/datasets/wcukierski/enron-email-dataset)

**Extra documentation**:
*   [POS](https://universaldependencies.org/u/pos/)
*   [SpaCy - English](https://spacy.io/models/en#en_core_web_sm)

In [ ]:
# ---- LIBRARIES ----
import email
import kagglehub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import spacy

from enum import Enum
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.language import Language
from tqdm.notebook import tqdm


# init spacy
flag_gpu = spacy.prefer_gpu()
print(f"GPU available:{flag_gpu}")


# init tpqdm
tqdm.pandas()

In [2]:
# ---- FUNCTIONS ----
def extract_fields(msg :email.message, fields :list[str]|None=None) -> dict[str, str]:
    """
    This function extracts the required fields from the given email message.
    If no field is not specified, all the available ones are returned.

    Parameters
    ----------
    msg : email.message
        Email message.
    fields : list | None, optional
        The fields to extract, by default ``None``.

    Returns
    -------
    :dict[str, str]
        A dictionary ``{field_name:field_value}`` containing the fields extracted is returned.

    Notes
    -----
    - If  ``fields`` is ``None``, all the available fields are extracted.
    - Fields must be in lower case.
    """
    res = dict()

    if fields is None:
        res = {str.lower(key):msg[key] for key in msg.keys()}
    else:
        res = {str.lower(key):msg[key] for key in msg.keys() if str.lower(key) in fields}

    if fields is None or "content" in fields:
        parts = list()
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                parts.append(part.get_payload())
        res["content"] = "".join(parts)
    return res


def identity_tokenizer(text):
    """
    Returns ``text`` as it is.
    """
    return text


def split_email_addresses(addresses :str) -> list[str]:
    """
    Extracts the email addresses contained in the provided string.

    Parameters
    ----------
    addresses : str
        String containing email addresses.

    Returns
    -------
    :list[str]
        List of strings that are the email addresses found.

    Notes
    -----
    - If ``addresses`` is not a string, an empty list is returned.
    """
    if isinstance(addresses, str):
        addrs = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', addresses)
    else:
        addrs = list()
    return addrs

In [24]:
# ---- CLASSES ----
class Preprocessor():
    """
    This class implements a simple preprocessor to split documents into lists of tokens.
    It is mainly based on spaCy.
    """

    class CREIL(Enum):
        """
        Common regex of invalid lines.
        """
        HEADER = r"^\s*(?:-{5,}|.*-{5,}|from:|to:|cc:|ccn:|sent by:|subject:).*$"
        """
        Removes (forwarded, etc.) email header and blank lines.
        """

    class CREIT(Enum):
        """
        Common regex of invalid tokens.
        """
        ALPHA_2 = r"^(?:.*[^a-z_]{1,}.*|.)$"
        """
        Only tokens of two or more characters consisting only of alphabetic characters are retained.
        """
        ALPHANUM = r".*(?:\W|_).*"
        """
        Removes tokens that contain non-alphanumeric characters.
        """


    @Language.component("lower_case_lemmas")
    def _lower_case_lemmas(doc :spacy.tokens.Doc) -> spacy.tokens.Doc:
        """
        Changes the capitalization of the lemmas to lowercase.

        Parameters
        ----------
        doc : spacy.tokens.Doc
            Doc to modify.

        Returns
        -------
        :spacy.tokens.Doc
            Doc modified.
        """
        for token in doc :
            token.lemma_ = token.lemma_.lower()
        return doc


    def __init__(
        self,
        case_sensitive :bool=True,
        lemmatize :bool=False,
        noun_chunks :bool=False,
        keep_stopwords :bool=True,
        extend_stopwords :set[str]=None,
        regex_flags :int=0,
        regex_invalid_line :'str|Preprocessor.CREIL'=None,
        regex_invalid_tokens :'str|Preprocessor.CREIT'=None,
        ent_to_keep :set[str]=None,
        ent_to_rm :set[str]=None,
        pos_to_keep :set[str]=None,
        pos_to_rm :set[str]=None,
    ):
        """
        Parameters
        ----------
        case_sensitive : bool, optional
            If ``False``, all returned tokens are lowercase, by default ``True``.
        lemmatize : bool, optional
            If ``True``, tokens are lemmatized, by default ``False``.
        noun_chunks : bool, optional
            If ``True``, chunk nouns are merged, by default ``False``.
        keep_stopwords : bool, optional
            If ``True``, stop words are kept, by default ``False``.
        extend_stopwords : set[str], optional
            List of stop words to add, by default ``None``.
        regex_flags : int, optional
            Flags of the regular expressions, by default ``0``.
        regex_invalid_line : str|Preprocessor.CREIL, optional
            Regex to mark lines that must be excluded, by default ``None``.
        regex_invalid_tokens : str|Preprocessor.CREIT, optional
            Regex to mark tokens that must to be excluded, by default ``None``.
        ent_to_keep : set[str], optional
            Only tokens marked with one of these entity tags are kept,
            by default all tokens are retained.
        ent_to_rm : set[str], optional
            Tokens marked with one of these entity tags are removed,
            by default all tokens are retained.
        pos_to_keep : set[str], optional
            Only tokens marked with one of these POS tags are kept,
            by default all tokens are retained.
        pos_to_rm : set[str], optional
            Tokens marked with one of these POS tags are removed,
            by default all tokens are retained.
        """
        self.nlp = spacy.load('en_core_web_sm')

        # update nlp piepline
        if not lemmatize and not noun_chunks:
            self.nlp.remove_pipe("lemmatizer")
            if ent_to_keep is None and ent_to_rm is None and pos_to_keep is None and pos_to_rm is None:
                self.nlp.remove_pipe("tagger")
                self.nlp.remove_pipe("parser")
                self.nlp.remove_pipe("attribute_ruler")
                self.nlp.remove_pipe("ner")
        if noun_chunks:
            self.nlp.add_pipe("merge_noun_chunks")
        if lemmatize and not case_sensitive:
            self.nlp.add_pipe("lower_case_lemmas", name="lower_case_lemmas")

        # standard regex
        self.regex_empty_line = r"^\s*$"
        self.regex_endline_1 = r"(=20\n)"
        self.regex_endline_2 = r"(=\n)"

        self.pattern_empty_line = re.compile(self.regex_empty_line, regex_flags)
        self.pattern_endline_1 = re.compile(self.regex_endline_1, regex_flags)
        self.pattern_endline_2 = re.compile(self.regex_endline_2, regex_flags)

        # save fields
        self.case_sensitive = case_sensitive
        self.noun_chunks = noun_chunks
        self.lemmatize = lemmatize
        self.keep_stopwords = keep_stopwords
        self.extend_stopwords = extend_stopwords
        self.regex_flags = regex_flags
        self.regex_invalid_line = regex_invalid_line.value if isinstance(regex_invalid_line, Preprocessor.CREIL) else regex_invalid_line
        self.regex_invalid_tokens = regex_invalid_tokens.value if isinstance(regex_invalid_tokens,  Preprocessor.CREIT) else regex_invalid_tokens
        self.ent_to_keep = ent_to_keep
        self.ent_to_rm = ent_to_rm
        self.pos_to_keep = pos_to_keep
        self.pos_to_rm = pos_to_rm

        # assemble the condition that tokens/lines must satisfy
        self.conditions = list()
        if not keep_stopwords:
            self.conditions.append(lambda x: not x.is_stop)
            if extend_stopwords is not None:
                if lemmatize:
                    self.conditions.append(lambda x: x.lemma_ not in extend_stopwords)
                else:
                    self.conditions.append(lambda x: x.text not in extend_stopwords if case_sensitive else x.text.lower() not in extend_stopwords)
        if ent_to_keep is not None:
            self.conditions.append(lambda x: x.ent_type_ in ent_to_keep)
        if ent_to_rm is not None:
            self.conditions.append(lambda x: x.ent_type_ not in ent_to_rm)
        if pos_to_keep is not None:
            self.conditions.append(lambda x: x.pos_ in pos_to_keep)
        if pos_to_rm is not None:
            self.conditions.append(lambda x: x.pos_ in pos_to_rm)

        if regex_invalid_line is not None:
            self.pattern_invalid_line = re.compile(self.regex_invalid_line, regex_flags)
        if regex_invalid_tokens is not None:
            self.pattern_invalid_tokens = re.compile(self.regex_invalid_tokens, regex_flags)
            self.conditions.append(lambda x: not self.pattern_invalid_tokens.match(x.text))
        return


    def preprocess(self, document :str) -> list[str]:
        """
        Applies the preprocessing procedure to the given document.

        Parameters
        ----------
        document : str
            Document to process.

        Returns
        -------
        :list[str]
            List of tokens representing the document.
        """
        # MIME protocol for base64, the max line length in the encoded data is 76 characters
        # an '=' sign at the end of an encoded line is used to tell the decoder that the line is to be continued,
        # while an '=20' sign is used to tell the line ends
        document = self.pattern_endline_1.sub("\n", document)
        document = self.pattern_endline_2.sub("", document)

        # read the document line by line and remove the invalid ones that are:
        # - blank lines
        # - lines that match the given regex <self.regex_invalid_line>
        # - lines preceded by invalid non-white lines
        prev = True
        filtered_document = list()
        for line in document.splitlines():
            if self.pattern_empty_line.match(line):
                prev = True
            elif prev and (self.regex_invalid_line is None or not self.pattern_invalid_line.match(line)):
                filtered_document.append(line)
            else:
                prev = False

        # replace all withespaces with a single whitespace
        document = re.sub(r"\s{1,}", " ", "\n".join(filtered_document))

        # tokenize, etc. using spaCy
        doc = self.nlp(document)
        if self.lemmatize:
            tokens = [tk.lemma_ for tk in doc if all(f(tk) for f in self.conditions)]
        else:
            if self.case_sensitive:
                tokens = [tk.text for tk in doc if all(f(tk) for f in self.conditions)]
            else:
                tokens = [tk.text.lower() for tk in doc if all(f(tk) for f in self.conditions)]
        return tokens



class KMeansHelper():
    """
    This class is based on the module ``sklearn.cluster.KMeans`` and
    provides some heuristicher to estimate the best number of clusters.
    """

    def __init__(
        self,
        init :str="k-means++",
        n_init :int|str="auto",
        random_state :int=None
    ):
        """
        Parameters
        ----------
        init : str, optional
            Method for initialization, by default ``'k-means++'``:
            - ``'k-means++'``: selects initial cluster centroids using sampling based on an empirical probability distribution
            of the points' contribution to the overall inertia. This technique speeds up convergence.
            The algorithm implemented is greedy k-means++. It differs from the vanilla k-means++ by making several trials
            at each sampling step and choosing the best centroid among them.
            - ``'random'``: choose n_clusters observations at random from data for the initial centroids.
        n_init : str, optional
            Number of times the k-means algorithm is run with different centroid seeds.
            By default ``n_init='auto'``, the number of runs depends on the value of init:
            10 if using ``init='random'``; 1 if using ``init='k-means++'``.
        random_state : int, optional
            Determines random number generation for centroid initialization. Use an int to make the randomness deterministic,
            by default ``None``.
        """
        self.init = init
        self.n_init = n_init
        self.random_state = random_state
        return


    def elbow_method(self, X :np.ndarray, min_K :int, max_K :int) -> dict[int, int]:
        """
        Returns and draws the inertia as the number of clusters changes.
        This heuristic is used to determine the number of clusters in a data set ``X``.

        Parameters
        ----------
        X : np.ndarray
            New data to transform.
        min_K : int
            Minimum number of clusters.
        max_K : int
            maximum number of clusters.

        Returns
        -------
        :dict[int, int]
            A dictionary containing the SSE for every possible number of clusters
            between ``min_K`` and ``max_K``.
        """
        sse = {}
        for k in range(min_K, max_K):
            kmeans = KMeans(n_clusters=k, init=self.init, n_init=self.n_init, random_state=self.random_state).fit(X)
            sse[k] = kmeans.inertia_

        plt.figure()
        plt.plot(list(sse.keys()), list(sse.values()), marker='*')
        plt.title("Elbow method")
        plt.xlabel("Number of cluster (K)")
        plt.ylabel("SSE")
        plt.show()
        return sse


    def fit_predict(self, X :np.ndarray, K :int) -> tuple[np.ndarray, np.ndarray]:
        """
        Compute cluster centers and predict cluster index for each sample.

        Parameters
        ----------
        X : np.ndarray
            New data to transform.
        K : int
            Number of clusters.

        Returns
        -------
        :tuple[np.ndarray, np.ndarray]
            Index of the cluster each sample belongs to and the centroids
        """
        kmeans = KMeans(n_clusters=K, init=self.init, n_init=self.n_init, random_state=self.random_state)
        clusters = kmeans.fit_predict(X)
        centroids = kmeans.cluster_centers_
        return clusters, centroids


    def silhouette_score(self, X :np.ndarray, min_K :int, max_K :int) -> dict[int, int]:
        """
        Returns the silhouette score [-1, 1] as the number of clusters changes.
        This heuristic is used to determine the number of clusters in a data set ``X``.

        Parameters
        ----------
        X : np.ndarray
            New data to transform.
        min_K : int
            Minimum number of clusters.
        max_K : int
            maximum number of clusters.

        Returns
        -------
        :dict[int, int]
            A dictionary containing the silhouette score for every possible number of clusters
            between ``min_K`` and ``max_K``.
        """
        sc = {}
        print("Number of clusters:")
        for k in range(min_K, max_K):
            kmeans = KMeans(n_clusters=k, init=self.init, n_init=self.n_init, random_state=self.random_state).fit(X)
            sc[k] = silhouette_score(X, kmeans.labels_, metric='euclidean')
            print(f" - K:{k} => Silhouette coeff.: {sc[k]}")
        return sc

## Corpus

In [ ]:
# ---- DATASET ----
# download latest version
path = kagglehub.dataset_download("wcukierski/enron-email-dataset")
print(f"Path to dataset files: {path}")

# read the dataset
df = pd.read_csv(path + "/emails.csv")

# from string to email.message
df["message"] = df["message"].apply(lambda x: email.message_from_string(x))

# extract main fields
df = df.join(
    df.apply(
        lambda row: extract_fields(row.message, ["message-id", "date", "from", "to", "subject", "content"]),
        axis='columns',
        result_type='expand'
    )
)

# drop unused column and duplicated messages
df.drop(columns=["message"], inplace=True)
df.drop_duplicates(["content"], keep="first", inplace=True)

# split addresses into a list of strings
df["from"] = df["from"].apply(
    lambda x: split_email_addresses(x),
)
df["to"] = df["to"].apply(
    lambda x: split_email_addresses(x),
)

In [ ]:
# minimum and maximum length (in characters) of emails to be analyzed
min_len, max_len = 600, 610
# percentage (0, 1] of emails that must be analyzed
sample_frac = 1.

corpus = df[df["content"].apply(lambda x: len(x)).between(min_len, max_len)].sample(frac=sample_frac)

display(corpus)
print(f"\nRandom exemple =>\n{corpus.sample(1).iloc[0, 6]}")

## Tokenizer

In [ ]:
pp = Preprocessor(
    case_sensitive = False,
    lemmatize = True,
    noun_chunks = False,
    keep_stopwords = False,
    extend_stopwords = {"email", "mail", "hereto", "link", "pm", "recipient", "sender", "thank", "thanks", "time"},
    regex_flags = re.IGNORECASE,
    regex_invalid_line = Preprocessor.CREIL.HEADER,
    regex_invalid_tokens = Preprocessor.CREIT.ALPHA_2,
    pos_to_keep = {"NOUN"},
    ent_to_rm = {"DATE", "TIME"}
)

tokenized_corpus = list(corpus["content"].progress_apply(pp.preprocess))

## TF-IDF

In [ ]:
# skip tokens that do not have a relative document frequency in this range
min_df = 0.01
max_df = 1.00

tfidf_vectorizer =  TfidfVectorizer(tokenizer=identity_tokenizer, token_pattern=None, lowercase=False, min_df=min_df, max_df=max_df)
tfidf_corpus = tfidf_vectorizer.fit_transform(tokenized_corpus)

tfidf_df = pd.DataFrame(tfidf_corpus.toarray(), index=corpus.index, columns=tfidf_vectorizer.get_feature_names_out())
display(tfidf_df)

## KMeans with TF-IDF

In [ ]:
# use elbow method and silhouette score to estimate the number of clusters
kmeans = KMeansHelper()

_ = kmeans.elbow_method(tfidf_corpus, 2, 12)
_ = kmeans.silhouette_score(tfidf_corpus, 2, 12)

In [ ]:
# number of clusters
K = 6
# number of tokens with the heighest tf-idf score to show per cluster
W = 5

clusters, centroids = kmeans.fit_predict(tfidf_corpus, K)

# for each cluster, sort the token indexes according to their tf-idf score (descending order)
argsort_tokens = (np.argsort(centroids, axis=1)[:, ::-1])
# find the index of W tokens with the highest tf-idf score that are not among
# W tokens with the highest tf-idf score in other clusters
unique_main_tk_idxs = np.full((K, W), -1)

for k in range(K):
    counter = 0
    for tk_idx in argsort_tokens[k]:
        if tk_idx not in np.delete(argsort_tokens[:, :W], k, axis=0) and tk_idx not in unique_main_tk_idxs:
            unique_main_tk_idxs[k, counter] = tk_idx
            counter += 1
        if counter == W:
            break

    main_tokens = tfidf_vectorizer.get_feature_names_out()[argsort_tokens[k, :W]]
    unique_main_tokens = tfidf_vectorizer.get_feature_names_out()[unique_main_tk_idxs[k]]
    idx_docs_in_cluster = np.flatnonzero(clusters == k)

    print(f"Cluster {k} =>")
    print(f" - size: {len(idx_docs_in_cluster)}")
    print(f" - main tokens: {main_tokens}")
    print(f" - unique main tokens: {unique_main_tokens}")
    print(f" - random samples: {np.random.choice(idx_docs_in_cluster, 3)}")
print()

# documents and centroids as points in a n-dimensional space
points_nD = np.concatenate((centroids, tfidf_corpus.toarray()))

# t-SNE to reduce the number of dimensions
tsne = TSNE(n_components=2, perplexity=25)
points_2D = tsne.fit_transform(points_nD)

# plot the clusters with their centroids
fig, ax = plt.subplots()
scatter_docs = ax.scatter(points_2D[K:, 0], points_2D[K:, 1], c=clusters, alpha=0.5)
scatter_cens = ax.scatter(points_2D[:K, 0], points_2D[:K, 1], c=["red"], alpha=1.0, marker='^', label="Centroids")
legend_docs = ax.legend(*scatter_docs.legend_elements(), loc="lower right", title="Clusters")
legend_cens = ax.legend(handles=[scatter_cens], loc="upper left")
ax.add_artist(legend_docs)
ax.add_artist(legend_cens)
plt.title("t-SNE: KMeans with TF-IDF vectors")
plt.show()

## Word2Vect